<a href="https://colab.research.google.com/github/Mani1213/Resume_shortlisting/blob/main/Resume_shortlisting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install -q openai langchain
pip install --upgrade pillow
pip install --upgrade fastai pdfplumber pikepdf torchvision
pip install --upgrade torch==2.2.0
pip install --upgrade torchaudio torchdata torchtext
pip install -q "unstructured[pdf]"
pip install -q chromadb
pip install -q tiktoken
pip install -q langchain_experimental
pip install -q pip install wikipedia

In [2]:
import os
import openai
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [3]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%bash
pip install -q kaggle
mkdir ~/.kaggle
cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json
kaggle datasets download -d snehaanbhawal/resume-dataset
mkdir cv_read
unzip /content/resume-dataset.zip -d cv_read


Archive:  /content/resume-dataset.zip
  inflating: cv_read/Resume/Resume.csv  
  inflating: cv_read/data/data/ACCOUNTANT/10554236.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/10674770.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/11163645.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/11759079.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/12065211.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/12202337.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/12338274.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/12442909.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/12780508.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/12802330.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/13072019.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/13130984.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/13294301.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/13491889.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/13701259.pdf  
  inflating: cv_read/data/data/ACCOUNTANT/14055988.pdf  
  inflat

100%|██████████| 62.5M/62.5M [00:00<00:00, 81.5MB/s]


### Load documents from a folder consisting of multiple resume PDF file using DirectoryLoader

In [7]:
directory = '/content/cv_read/data/data/ENGINEERING/'
def load_docs(directory):
  loader = DirectoryLoader(directory,show_progress=True) #unstructuredloader by default has used this auto identify file type and load it, mode="single", strategy='fast'(other option is strategy='hi_res' that use yolo varient if mode is elements)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)


  0%|          | 0/118 [00:00<?, ?it/s]ERROR:langchain_community.document_loaders.directory:Error loading file /content/cv_read/data/data/ENGINEERING/30288581.pdf


ValueError: unstructured package not found, please install it with `pip install unstructured`

  1%|          | 1/118 [01:12<2:20:38, 72.12s/it]

In [ ]:
documents=documents[:10]

*  Split the documents using RecursiveCharacterTextSplitter
  with chunk size = 1000, chunk overlap=100
* Set the environment with OPENAI_API_KEY
* Create OpenAIEmbeddings instance



In [ ]:
def split_docs(documents, chunk_size=1000, chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=userdata.get('OPENAI_API_KEY'))

78


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


* Let’s use Chroma DB from langchain vector stores to embed and store the vector database on local hard disk
* Now load the persisted database from disk
* Initiate the Chroma vector database retriever with k=2, where default search type is similarity_score_threshold
* The above retriever helps us to retrieve document section from PDF’s which is suitable for give role

In [ ]:
from langchain.vectorstores import Chroma
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=docs,
                                 embedding=embedding,
                                 persist_directory=persist_directory)
# persiste the db to disk
vectordb.persist()
vectordb = None

# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

retriever = vectordb.as_retriever(search_kwargs={"k": 2}) # by default search_type="similarity_score_threshold"

* One of the resumes loaded before had skills and other information related to Machine learning role, so it extracted contents of that resume

In [ ]:
docs = retriever.invoke("which candidate is good fit for Machine learning engineer roles.")
print(docs[0].page_content)

Work Experience Engineering Team Lead Mar 2011 to Current Company Name ï¼​ City , State

Coordinate the product road-map through facilitating planning meetings and preparing feature-sets. Prepare technical architecture proposals for enhancements and integration of existing third party software systems. Design, document and execute engineering procedures, including customization delivery, escalation and technical modernization enhancements. Coach and mentor individuals on principles of software development.

Software Engineer Nov 2008 to Apr 2011 Company Name ï¼​ City , State

Responsible for research, design and development of enhancements to web based applications. Ensure agile software development principles are followed through coaching and mentoring development teams.

Web Developer Nov 2006 to Dec 2008 Company Name ï¼​ City , State

Develop and maintain web-based applications.

Jr. Software Engineer Dec 2005 to Dec 2006 Company Name ï¼​ City , State


* One of the resumes loaded before had skills and other information related to Data Analyst role, so it extracted contents of that resume

In [ ]:
docs = retriever.invoke("Give name of candidate who is good fit for Data Analyst roles.")

print(docs[0].page_content)

To obtain a rewarding position in a field where my skills can be further developed and utilized. Summary of Qualifications Three years of experience in customer service Performed volunteer work within the community through various service organizations Computer literate and a dedicated team player with a strong work ethic Dependable, motivated, detail oriented individual with strong organizational skills Excellent verbal and written communication skills

Highlights

Experienced with Microsoft Office - Word, Excel, Outlook, SharePoint & Costpoint Administrative Assistant Benefits Clerical Clients Data entry Database Features Filing Forms GIS Mailing Meetings

Excel Mail Outlook SharePoint Microsoft Office - Word Office equipment Policies Processes Reception Recording Retail Scanning Telephone Phone Typing

Skills


* Create a ChatOpenAI instance with temperature=0 with model as ‘gpt-3.5-turbo’
* Initiate a RetrievalQA chain with ChatOpenAI instance created in last step and with following config : chain_type=”stuff”(means we just stuff entire data into context to pass to LLM. But other methods include Map_reduce,Refine and Map_rerank), retriever (created in previous steps)

In [ ]:
from langchain.chat_models import ChatOpenAI
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

from langchain.chains import RetrievalQA
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


* Write a function to post process LLM response, Usually in the response of LLM it has meta data i.e source of a document from it answered. So we should capture that detail and print it.

In [ ]:
warning = "If you don't know the answer, just say that you don't know, don't try to make up an answer"
job_description = "MS or PhD in computer science or a related technical field,5+ years of industry work experience. Good sense of product with a focus on shipping user-facing data-driven features, Expertise in Python and Python based ML/DL and Data Science frameworks. \
Excellent coding, analysis, and problem-solving skills. Proven knowledge of data structure and algorithms. \
Familiarity in relevant machine learning frameworks and packages such as Tensorflow, PyTorch and HuggingFace\
Experience working with Product Management and decomposing feature requirements into technical work items to ship products\
Experience with generative AI, knowledge of ML Ops and ML services is a plus. This includes Pinecone, LangChain, Weights and Biases etc. \
Familiarity with deployment technologies such as Docker, Kubernetes and Triton are a plus\
Strong communication and collaboration skills"
question = warning+job_description + " Based on the given job description"
query = question + "short list resumes which is good fit based on skills,education and work experience mwntioned in it? also provide the candidate name which will be mentioned in first line of pdf without subheading"
# query = "short list resumes which is good fit for Data analysis roles based on skills,education and work experience mwntioned in it?"

llm_response = qa_chain(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Based on the given job description, the following candidate seems to be a good fit:

Candidate Name: Not provided in the given context

Qualifications:
- MS or PhD in computer science or a related technical field
- 5+ years of industry work experience
- Good sense of product with a focus on shipping user-facing data-driven features
- Expertise in Python and Python based ML/DL and Data Science frameworks
- Excellent coding, analysis, and problem-solving skills
- Proven knowledge of data structure and algorithms
- Familiarity in relevant machine learning frameworks and packages such as Tensorflow, PyTorch, and HuggingFace
- Experience working with Product Management and decomposing feature requirements into technical work items to ship products
- Experience with generative AI, knowledge of ML Ops and ML services is a plus (Pinecone, LangChain, Weights and Biases, etc.)
- Familiarity with deployment technologies such as Docker, Kubernetes, and Triton are a plus
- Strong communication and 

* Now lets create a prompt which contains a warning so LLM would not hallucinate, job description of some job from LinkedIn and message to short list resumes which is good fit based on skills, education and work experience mentioned in it
* Finally run function returned from LLM to retrieve pdf name(i.e resume file name)

# Retrieval QA chain.

In [ ]:
# full example
warning = "If you don't know the answer, just say that you don't know, don't try to make up an answer"
job_description = "MS or PhD in computer science or a related technical field,5+ years of industry work experience. Good sense of product with a focus on shipping user-facing data-driven features, Expertise in Python and Python based ML/DL and Data Science frameworks. \
Excellent coding, analysis, and problem-solving skills. Proven knowledge of data structure and algorithms. \
Familiarity in relevant machine learning frameworks and packages such as Tensorflow, PyTorch and HuggingFace\
Experience working with Product Management and decomposing feature requirements into technical work items to ship products\
Experience with generative AI, knowledge of ML Ops and ML services is a plus. This includes Pinecone, LangChain, Weights and Biases etc. \
Familiarity with deployment technologies such as Docker, Kubernetes and Triton are a plus\
Strong communication and collaboration skills"
question = warning+job_description + " Based on the given job description"
query = question + "short list resumes which is good fit based on skills,education and work experience mwntioned in it? also provide the candidate name which will be mentioned in first line of pdf without subheading"
# query = "short list resumes which is good fit for Data analysis roles based on skills,education and work experience mwntioned in it?"

llm_response = qa_chain(query)
process_llm_response(llm_response)

Based on the given job description, it is not possible to shortlist resumes or provide candidate names without subheadings. The provided information does not include specific names or subheadings for resumes.


Sources:
/content/cv_read/data/data/ENGINEERING/12011623.pdf
/content/cv_read/data/data/ENGINEERING/12011623.pdf


# Retrieving resume content which is suitable based on given job description

In [ ]:
warning = "If you don't know the answer, just say that you don't know, don't try to make up an answer"
job_description = "MS or PhD in computer science or a related technical field,5+ years of industry work experience. Good sense of product with a focus on shipping user-facing data-driven features, Expertise in Python and Python based ML/DL and Data Science frameworks. \
Excellent coding, analysis, and problem-solving skills. Proven knowledge of data structure and algorithms. \
Familiarity in relevant machine learning frameworks and packages such as Tensorflow, PyTorch and HuggingFace\
Experience working with Product Management and decomposing feature requirements into technical work items to ship products\
Experience with generative AI, knowledge of ML Ops and ML services is a plus. This includes Pinecone, LangChain, Weights and Biases etc. \
Familiarity with deployment technologies such as Docker, Kubernetes and Triton are a plus\
Strong communication and collaboration skills"
question = warning+job_description + " Based on the given job description"
query = question + "retrive the full document information of a resume which is good fit based on skills,education and work experience mwntioned in it? "

resume_doc = retriever.invoke(query)

resume_doc = resume_doc[0].page_content
print(resume_doc)

Design and implement statistical / predictive models and cutting edge algorithms utilizing diverse sources of data to predict Diversified experience with Engineering, Manufacturing, Retailing, Higher Education and IT /Data related companies Fascinated by learning cutting edge technologies, such as; Data Mining and Machine Learning Handled a team of 4 during my 3 years of Tech experience and managed more than 200 students and effectively evaluated the performance of each student and utilize assessment methods to judge overall progress during my teaching experience. Qualifications


* Demonstrating the use of prompt template:
  Define a prompt template to extract skills, education, projects, publications, work experience as comma separated python list.

In [ ]:
review_template = """\
For the following text, extract the following information:

Skills: what are the technical and non technical skills? \
Answer output them as a comma separated Python list.

Education: What is the highest education of the candidate and what is the GPA as mentioned in the text?\
Answer Output should be the university/college name and GPA if given in text, output them as a comma separated Python list.

Projects: Extract all project titles mentioned in a text\
and output them as a comma separated Python list.

Publications: Extract all publication titles mentioned in a text\
and output them as a comma separated Python list.

Work experience: Extract all organisation name where he/she has worked along with number of years or months worked there and also extract designation\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
Skills
Education
Projects
Publications
Work experience

text: {text}
"""

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\nSkills: what are the technical and non technical skills? Answer output them as a comma separated Python list.\n\nEducation: What is the highest education of the candidate and what is the GPA as mentioned in the text?Answer Output should be the university/college name and GPA if given in text, output them as a comma separated Python list.\n\nProjects: Extract all project titles mentioned in a textand output them as a comma separated Python list.\n\nPublications: Extract all publication titles mentioned in a textand output them as a comma separated Python list.\n\nWork experience: Extract all organisation name where he/she has worked along with number of years or months worked there and also extract designationand output them as a comma separated Python list.\n\nFormat the output as JSON with the following ke

##**Create a conversation chain with memory**
Memory type used here is ConversationBufferWindowMemory

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.load_memory_variables({})

turbo_llm_memory = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)


memory_llm_conversation = ConversationChain(
    llm=turbo_llm_memory,
    memory = memory,
    verbose=True
)

messages = prompt_template.format_messages(text=resume_doc)
# chat = ChatOpenAI(temperature=0.0, model=turbo_llm_memory)
response = memory_llm_conversation(messages)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Not much, just hanging
AI: Cool
Human: [HumanMessage(content="For the following text, extract the following information:\n\nSkills: what are the technical and non technical skills? Answer output them as a comma separated Python list.\n\nEducation: What is the highest education of the candidate and what is the GPA as mentioned in the text?Answer Output should be the university/college name and GPA if given in text, output them as a comma separated Python list.\n\nProjects: Extract all project titles mentioned in a textand output them as a comma separated Python list.\n\nPublications: Extract all publication titles mentioned in a textand output them as a comma sep

Here, resume_doc assigned to text is a document retrieved in previous steps based on the job description.

In [ ]:
print(response.keys())
print(response['response'])

dict_keys(['input', 'history', 'response'])
I'm sorry, but I cannot extract the information you requested from the given text.


* recheck above if it is different from pdf
* Parsing the LLM output using output parser
Define response schemas for skills, projects and work experience that need to be extracted, using which create structured output parser.

Create a prompt template to instruct to extract skills, projects and work experience, which also include placeholder for context(I.e resume in our case) and format instructions(derived from StructuredOutputParser).

Now send the formatted prompt template to LLM whose output will be dictionary thus we can easily retrieve required answers using specific keys.

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

skills_schema = ResponseSchema(name="Skills",
                             description="what are the technical and non technical skills? \
Answer output them as a comma separated Python list.")



Projects_schema = ResponseSchema(name="Projects",
                                    description="Extract all project titles mentioned in a text\
and output them as a comma separated Python list.")


Work_experience_schema = ResponseSchema(name="Work experience",
                                    description="Extract all organisation name where he/she has worked along with number of years or months worked there and also extract designation\
and output them as a comma separated Python list.")

response_schemas = [skills_schema,

                    Projects_schema,

                   Work_experience_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Skills": string  // what are the technical and non technical skills? Answer output them as a comma separated Python list.
	"Projects": string  // Extract all project titles mentioned in a textand output them as a comma separated Python list.
	"Work experience": string  // Extract all organisation name where he/she has worked along with number of years or months worked there and also extract designationand output them as a comma separated Python list.
}
```


In [ ]:
review_template_2 = """\
For the following text, extract the following information:

Skills: what are the technical and non technical skills? \
Answer output them as a comma separated Python list.

Projects: Extract all project titles mentioned in a text\
and output them as a comma separated Python list.

Work experience: Extract all organisation name where he/she has worked along with number of years or months worked there and also extract designation\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
Skills
Projects
Work experience

text: {text}

{format_instructions}
"""


prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=resume_doc,
                                format_instructions=format_instructions)
print(messages[0].content)

For the following text, extract the following information:

Skills: what are the technical and non technical skills? Answer output them as a comma separated Python list.

Projects: Extract all project titles mentioned in a textand output them as a comma separated Python list.

Work experience: Extract all organisation name where he/she has worked along with number of years or months worked there and also extract designationand output them as a comma separated Python list.

Format the output as JSON with the following keys:
Skills
Projects
Work experience

text: [Document(page_content='Design and implement statistical / predictive models and cutting edge algorithms utilizing diverse sources of data to predict Diversified experience with Engineering, Manufacturing, Retailing, Higher Education and IT /Data related companies Fascinated by learning cutting edge technologies, such as; Data Mining and Machine Learning Handled a team of 4 during my 3 years of Tech experience and managed more t

In [ ]:
response2 = turbo_llm_memory(messages)

print(response2.content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


```json
{
	"Skills": "Data Mining, Machine Learning, R, SAS, Python, Pandas, Web scraping, Beautiful soup, Oracle, Microsoft SQL, ASP.NET, C#, HTML, JMP, SPSS, GIS, Tableau, ggplot, Random Forests, Decision Trees, Boosted Trees, Ridge, LASSO, Elastic Net regression models, PCA, Outlier analysis, Linear Regression, Clustering techniques, logistic regression",
	"Projects": "",
	"Work experience": "Engineering and Quality Technician Company Name, 3 years"
}
```


#**Demonstrating Sequential Chains :**

First chain is to extract technical and non technical skills.

Second chain is for what are the job roles among Data Scientist, Machine learning Engineer, Software Engineer, Data Engineer, Devops Engineer, Cloud Architect. Are suited based on the given skill sets.

Third chain is for explaining each skill as for what kind of projects are these useful.

In [ ]:
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI

import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo")

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

first_prompt = ChatPromptTemplate.from_template(
    "Skills: what are the technical and non technical skills? \
Answer output them as a comma separated Python list."
    "\n\n{resume_doc}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="skills"
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you name what the job roles among Data Scientist, Machine learning Engineer, Software Engineer, Data Engineer, Devops Engineer, Cloud Architect. Are suited based on the given skill sets"
    "\n\n{skills}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="job_titles"
                    )


third_prompt = ChatPromptTemplate.from_template(
    "Explain each skill as for what kind of projects are these usefull:\n\n{skills}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="skills_explanation"
                      )

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["resume_doc"],
    output_variables=["skills", "job_titles","skills_explanation"],
    verbose=True
)

seqchain_output = overall_chain(resume_doc)
seqchain_output




> Entering new SequentialChain chain...

> Finished chain.


{'resume_doc': 'Design and implement statistical / predictive models and cutting edge algorithms utilizing diverse sources of data to predict Diversified experience with Engineering, Manufacturing, Retailing, Higher Education and IT /Data related companies Fascinated by learning cutting edge technologies, such as; Data Mining and Machine Learning Handled a team of 4 during my 3 years of Tech experience and managed more than 200 students and effectively evaluated the performance of each student and utilize assessment methods to judge overall progress during my teaching experience. Qualifications',
 'skills': "['Design and implement statistical / predictive models', 'Cutting edge algorithms', 'Data mining', 'Machine learning', 'Team management', 'Performance evaluation', 'Assessment methods']",
 'job_titles': 'Based on the given skill sets, the job roles that are suited are as follows:\n\n- Data Scientist: Design and implement statistical / predictive models, cutting-edge algorithms, dat

# **Demonstrating LangChain Tools and Agents**

Agents : The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hard coded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

Tools: are interfaces that an agent can use to interact with the world.

Here we define one custom tool to return job descriptions, other inbuilt tools which I used was llm-math and Wikipedia.

query: what are the technical and non technical skills?

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool

@tool
def job_desription(text: str)-> str:
 """Returns job disriptions mentioned below, use this for any \
 questions related to knowing the job disription. \
 The input should always be an empty string, \
 and this function will always return a string containing job disriptions.\ """

 return "Job discriptions:\
 1)Machine learning Engineer:Machine Learning Engineer with expertise in designing and developing robust models and algorithms to solve complex business problems. Experienced in end-to-end machine learning pipelines, from data preprocessing to deployment. Proficient in Python, TensorFlow, and PyTorch. Skilled in data preprocessing, feature engineering, and cloud platforms (AWS, Azure, GCP). Strong communicator with a collaborative approach and a proven ability to drive projects to completion.\
 2) Computer Vision Engineer:Computer Vision Engineer specializing in 3D scan structure extraction and model development. Collaborates with product and research teams to enhance current products and enable new ones. Experienced with massive datasets, 2D Deep Learning, and Computer Vision using PyTorch and/or TensorFlow. Balances generalist and researcher roles, ensuring ML models transition into meaningful production. Works closely with product owners to deliver value efficiently to customers."


tools = load_tools(["llm-math","wikipedia"], llm=turbo_llm)

agent= initialize_agent(
    tools+ [job_desription],
    turbo_llm,  #turbo_llm, qa_chain,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

agent_template = """\
The following is the resume and query:

resume: {resume}

query: {query}
"""


prompt = ChatPromptTemplate.from_template(template=agent_template)
query_human = 'Skills: what are the technical and non technical skills? \Answer output them as a comma separated Python list.'
messages = prompt.format_messages(resume=resume_doc,
                                query=query_human)

messages


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


[HumanMessage(content='The following is the resume and query:\n\nresume: Design and implement statistical / predictive models and cutting edge algorithms utilizing diverse sources of data to predict Diversified experience with Engineering, Manufacturing, Retailing, Higher Education and IT /Data related companies Fascinated by learning cutting edge technologies, such as; Data Mining and Machine Learning Handled a team of 4 during my 3 years of Tech experience and managed more than 200 students and effectively evaluated the performance of each student and utilize assessment methods to judge overall progress during my teaching experience. Qualifications\n\nquery: Skills: what are the technical and non technical skills? \\Answer output them as a comma separated Python list.\n')]

# query: Give me the available job descriptions?

In [ ]:
agent_template = """\
The following is the resume and query:

resume: {resume}

query: {query}
"""


prompt = ChatPromptTemplate.from_template(template=agent_template)
query_human = 'Give me the available job discriptions?'
messages = prompt.format_messages(resume=resume_doc,
                                query=query_human)

result = agent(messages)



> Entering new AgentExecutor chain...
Could not parse LLM output: Thought: The user wants to know the available job descriptions.
Action:
```
job_desription("")
```
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: The job_desription function did not return a valid response. I need to fix the input and try again.
Action:
```
job_desription("")
```
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: The job_desription function is not returning a valid response. I need to check the function implementation and try again.
Action:
```
job_desription("")
```
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Could not parse LLM output: The job_desription function is still not returning a valid response. I need to check the function implementation and try again.
Action:
```
job_desription("")
```
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Could not parse LLM output: Th

# **Demonstrating Multi Prompt chain**

Demonstrating use the RouterChain paradigm to create a chain that dynamically selects the prompt to use for a given input. Specifically use the MultiPromptChain to create a question-answering chain that selects the prompt which is most relevant for a given question, and then answers the question using that prompt

# Define 3 prompt template:

## Job description:

In [ ]:
"""You are good at matching available job description with resume.\
Steps:\
1.Retrieve job descriptions from given tool attached with agent \
2.Compare if resume can be selected based on any job description, if yes then return that specific job description
3.If no job description matches the return None
Here is a resume:
{input}"""

## Portfolio :

In [ ]:
"""
You are good at finding portfolio link from the given resume and return that link to the user.If link not found return None.

Here is a question:
{input}"""

'\nYou are good at finding portfolio link from the given resume and return that link to the user.If link not found return None.\n\nHere is a question:\n{input}'

## Summary:

In [ ]:
"""
You are good at summerising the given resume. You will include skills, professional experience, education in the summary.

Here is a question:
{input}"""

'\nYou are good at summerising the given resume. You will include skills, professional experience, education in the summary. \n\nHere is a question:\n{input}'

In [ ]:
job_description_template = """
You are good at matching available job description with resume.\
Steps:\
1.Retreive job discriptions from givel tool attached with agent \
2.Compare if resume can be selected based on any job discription, if yes then retuen that specific job discription
3.If no job discription matches the return None

Here is a resume:
{input}"""


portfolio_finder_template = """

You are good at finding portfolio link from the given resume and return that link to the user.If link not found return None.

Here is a question:
{input}"""

summary_template = """
You are good at summerising the given resume. You will include skills, professional experience, education in the summary.

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "job_description",
        "description": "Good for providing job discription that is matched",
        "prompt_template": job_description_template
    },
    {
        "name": "portfolio",
        "description": "Good for returning portfolio link from resume",
        "prompt_template": portfolio_finder_template
    },
    {
        "name": "summary",
        "description": "Good for providing summary of resume",
        "prompt_template": summary_template
    }
]


## Create destination chains:

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    if name == "job_description":
        chain = agent
    elif name == "portfolio" :
        chain = LLMChain(llm=llm, prompt=prompt)
    else:
        chain = LLMChain(llm=llm, prompt=prompt)

    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
destinations

['job_description: Good for providing job discription that is matched',
 'portfolio: Good for returning portfolio link from resume',
 'summary: Good for providing summary of resume']

## Define multi prompt router prompt and multi prompt chain:

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )
chain

MultiPromptChain(verbose=True, router_chain=LLMRouterChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['input'], output_parser=RouterOutputParser(), template='Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the origi

## Let’s define a input prompt to for extracting portfolio link from resume:

In [ ]:
review_template1 = """\
For the following text, extract the following information:

Portfolio link: Extract portfolio link from given document.

text: {text}
"""


from langchain.prompts import ChatPromptTemplate

prompt_template1 = ChatPromptTemplate.from_template(review_template1)
messages1 = prompt_template1.format_messages(text=resume_doc[:]) #resume_doc[300:] to test portfolio link :None
print(prompt_template)



You are good at summerising the given resume. You will include skills, professional experience, education in the summary. 

Here is a question:
{input}


Let’s define a input prompt to give matching job_description, if nothing matches give None:

In [ ]:
review_template2 = """\
For the following text, extract the following information:

Give matching job_description, if nothing matches give None.

text: {text}
"""

from langchain.prompts import ChatPromptTemplate

prompt_template1 = ChatPromptTemplate.from_template(review_template2)
messages1 = prompt_template1.format_messages(text=resume_doc[:]) #resume_doc[300:] to test portfolio link :None
res=chain.run(messages1)
res

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
job_description: {'input': 'Design and implement statistical / predictive models and cutting edge algorithms utilizing diverse sources of data to predict Diversified experience with Engineering, Manufacturing, Retailing, Higher Education and IT /Data related companies Fascinated by learning cutting edge technologies, such as; Data Mining and Machine Learning Handled a team of 4 during my 3 years of Tech experience and managed more than 200 students and effectively evaluated the performance of each student and utilize assessment methods to judge overall progress during my teaching experience. Qualifications'}

> Entering new AgentExecutor chain...
Could not parse LLM output: I'm sorry, but I'm not sure what you're asking. Could you please clarify or provide more information?
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: The user's previous message seems to be a combination of a job description and qualification

ValueError: Missing some output keys: {'text'}

Not let’s create a response schema to extract portfolio link:

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

portfolio_link_schema = ResponseSchema(name="portfolio_link",
                             description="Give portfolio link from the given resume and return that link to the user\
Answer output them as a comma separated Python list.")

response_schemas = [portfolio_link_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"portfolio_link": string  // Give portfolio link from the given resume and return that link to the userAnswer output them as a comma separated Python list.
}
```


In [ ]:
review_template1 = """\
For the following text, extract the following information:

Portfolio link: Extract portfolio link from given document.

text: {text}

{format_instructions}
"""

from langchain.prompts import ChatPromptTemplate

prompt_template1 = ChatPromptTemplate.from_template(review_template1)
messages1 = prompt_template1.format_messages(text=res,format_instructions=format_instructions)

print(messages1[0].content)

NameError: name 'res' is not defined